#### RandonForest with LogisticRegression coefficient

In [32]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# NLP
import nltk
from soynlp.normalizer import *
from hanspell import spell_checker
from konlpy.tag import Okt

# ML
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split, ShuffleSplit, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score , recall_score, f1_score, roc_auc_score, roc_curve

#random
import random

#wordcloud
from wordcloud import WordCloud, STOPWORDS
import numpy as np
from PIL import Image

# 한글화
import matplotlib.pyplot as plt 
from matplotlib import rc
plt.rcParams['axes.unicode_minus'] = False #마이너스 부호 때문에 한글이 깨질 수가 있어 주는 설정
rc('font', family='Arial Unicode MS')
%matplotlib inline 

#time
import time

In [3]:
df_review = pd.read_csv('review_all_morph_tknized_rmved_v2.csv')
df_review = df_review[['date', 
          'tokenized_review', 
          'tokenized_removed_review', 
          'reviews_for_vectorize', 
          'noun_tokenized_review', 
          'noun_tokenized_removed_review', 
          'noun_reviews_for_vectorize', 
          'rating', 'label']]

df_review

,date,tokenized_review,tokenized_removed_review,reviews_for_vectorize,noun_tokenized_review,noun_tokenized_removed_review,noun_reviews_for_vectorize,rating,label
0,2023-09-10,"['상세', '설명', '에', '들어가지', '않아도', '검색', '결과', '...","['상세', '설명', '들어가지', '않아도', '검색', '결과', '중량', ...",상세 설명 들어가지 않아도 검색 결과 중량 가격 확인 되면 더 편리하겠습니다,"['상세', '설명', '검색', '결과', '중량', '가격', '바로', '확인...","['상세', '설명', '검색', '결과', '중량', '가격', '확인', '더']",상세 설명 검색 결과 중량 가격 확인 더,5,1.0
1,2023-09-09,"['필터', '기능', '잘', '되어있어', '편하게', '쇼핑', '함']","['필터', '기능', '잘', '되어있어', '편하게', '쇼핑', '함']",필터 기능 잘 되어있어 편하게 쇼핑 함,"['필터', '기능', '쇼핑', '함']","['필터', '기능', '쇼핑', '함']",필터 기능 쇼핑 함,5,1.0
2,2023-09-09,"['배송', '이', '빨라요', '그리고', '할인', '이', '많아요']","['배송', '빨라요', '할인', '많아요']",배송 빨라요 할인 많아요,"['배송', '할인']","['배송', '할인']",배송 할인,5,1.0
3,2023-09-09,"['50', '퍼', '쿠폰', '만원', '빵빵', '하게', '주는', '곳',...","['50', '퍼', '쿠폰', '만원', '빵빵', '하게', '주는', '곳',...",50 퍼 쿠폰 만원 빵빵 하게 주는 곳 처음 봄 주문 하면 6시간 만에 옴 앱 뭐,"['쿠폰', '만원', '빵빵', '곳', '처음', '봄', '주문', '옴', ...","['쿠폰', '만원', '빵빵', '곳', '처음', '봄', '주문', '옴', ...",쿠폰 만원 빵빵 곳 처음 봄 주문 옴 앱 뭐,5,1.0
4,2023-09-09,"['빠른', '배송']","['빠른', '배송']",빠른 배송,['배송'],['배송'],배송,5,1.0
...,...,...,...,...,...,...,...,...,...
11224,2016-03-02,"['모바일', '로', '쉽게', '살', '수', '있어서']","['모바일', '쉽게', '살', '있어서']",모바일 쉽게 살 있어서,"['모바일', '살', '수']","['모바일', '살']",모바일 살,5,1.0
11225,2016-03-02,"['아이', '를', '키우며', '좋은', '식', '자재', '에', '대한',...","['키우며', '좋은', '식', '자재', '대한', '갈증', '있는데', '이...",키우며 좋은 식 자재 대한 갈증 있는데 이렇게 건강하고 신선한 먹거리 쉽게 접 있...,"['아이', '식', '자재', '대한', '갈증', '먹거리', '접', '수',...","['식', '자재', '대한', '갈증', '먹거리', '접', '컴퓨터', '엄마...",식 자재 대한 갈증 먹거리 접 컴퓨터 엄마 앱 지금 처럼 식탁,5,1.0
11226,2016-03-02,"['아는', '동생', '소개', '로', '알', '게', '된', '마켓', '...","['아는', '동생', '소개', '알', '게', '된', '마켓', '컬리', ...",아는 동생 소개 알 게 된 마켓 컬리 늦게 게 아쉬울 정도 제품 퀄리티 완젼 좋고...,"['동생', '소개', '알', '마켓', '컬리', '안', '게', '정도', ...","['동생', '소개', '알', '마켓', '컬리', '게', '정도', '제품',...",동생 소개 알 마켓 컬리 게 정도 제품 퀄리티 완젼 포장 배송 눈 안보 사도 정말...,5,1.0
11227,2016-03-02,"['샛별', '배송', '정말', '훌륭합니다', '컬리', '라면', '무조건',...","['샛별', '배송', '정말', '훌륭합니다', '컬리', '라면', '무조건',...",샛별 배송 정말 훌륭합니다 컬리 라면 무조건 믿고 주문,"['샛별', '배송', '정말', '컬리', '라면', '무조건', '주문']","['샛별', '배송', '정말', '컬리', '라면', '무조건', '주문']",샛별 배송 정말 컬리 라면 무조건 주문,5,1.0


---

#### morph

In [76]:
#vectorize reviews
contents_for_vectorize_morph = df_review['reviews_for_vectorize']

#TfidfVectorizer
vectorizer_tfidf_morph = TfidfVectorizer(min_df=10, decode_error='ignore')

vectorizer_tfidf_morph.fit_transform(contents_for_vectorize_morph)

#말뭉치
corpus_morph = []

for word in vectorizer_tfidf_morph.get_feature_names_out():
    corpus_morph.append(word)

In [5]:
#RandomForest

start = time.time()

#feature, label 분리
X_morph = vectorizer_tfidf_morph.fit_transform(contents_for_vectorize_morph)
y_morph = df_review['label']

#train, test data 분리
X_morph_train, X_morph_test, y_morph_train, y_morph_test = train_test_split(X_morph, 
                                                                            y_morph, 
                                                                            test_size=0.3, 
                                                                            random_state=13, 
                                                                            stratify=y_morph)



#RandomForestClassifier
rdmforest_clf_morph = RandomForestClassifier(random_state=13, n_jobs=-1)

#GridSearch
params = {
    'max_depth' : [35],
    'n_estimators' : [400, 500],
    'min_samples_leaf' : [1], 
    'min_samples_split' : [6],
    'max_features' : [0.1]
}

grid_cv = GridSearchCV(estimator=rdmforest_clf_morph, 
                       param_grid=params, 
                       cv=5, 
                       n_jobs=-1)

grid_cv.fit(X_morph_train, y_morph_train)

print(start - time.time())

-10.045352935791016


In [6]:
grid_cv.best_estimator_

RandomForestClassifier(max_depth=35, max_features=0.1, min_samples_split=6,
                       n_estimators=400, n_jobs=-1, random_state=13)

In [7]:
#고려한 parameter
#'max_depth' : [20, 25, 30, 35]
#'n_estimators' : [100, 200, 300, 400, 450, 500]
#'min_samples_leaf' : [6, 8, 10, 12, 14]
#'min_samples_split' : [6,8]
#'max_features' : [0.1, 0.2,0.3, 0.5]

grid_cv.best_params_

{'max_depth': 35,
 'max_features': 0.1,
 'min_samples_leaf': 1,
 'min_samples_split': 6,
 'n_estimators': 400}

In [8]:
grid_cv.best_score_

0.8930025445292621

In [77]:
#X_test predict

#skfold
skfold = StratifiedKFold(n_splits=5)

#best randomforest
rdmforest_clf_morph = RandomForestClassifier(max_depth=35, 
                                             max_features=0.1, 
                                             min_samples_leaf=1, 
                                             min_samples_split=6, 
                                             n_estimators=400)
                                         

#평가지표
accuracy = []
f1 = []
auc = []


#test_data 평가지표
accuracy_test = []
f1_test = []
auc_test = []


for train_idx, validation_idx in skfold.split(X_morph_train, y=y_morph_train):
        
    X_train = X_morph_train[train_idx]
    X_validation = X_morph_train[validation_idx]
        
    y_train = y_morph_train.iloc[train_idx]
    y_validation = y_morph_train.iloc[validation_idx]
        
    #fit_predict
    rdmforest_clf_morph.fit(X_train, y_train)
    pred_validation = rdmforest_clf_morph.predict(X_validation)
        
        
    #accuracy, f1, auc
    accuracy.append(accuracy_score(pred_validation, y_validation))
    f1.append(f1_score(pred_validation, y_validation))
    auc.append(roc_auc_score(pred_validation, y_validation))

    
pred_test = rdmforest_clf_morph.predict(X_morph_test) 
        
#test_data accuracy, f1, auc
accuracy_test.append(accuracy_score(pred_test, y_morph_test))
f1_test.append(f1_score(pred_test, y_morph_test))
auc_test.append(roc_auc_score(pred_test, y_morph_test))
    
#평가지표

print('accuracy of sag : {}'.format(sum(accuracy) / len(accuracy)))
print('-'*100)
print('f1 of sag : {}'.format(sum(f1) / len(f1)))
print('-'*100)
print('auc of sag : {}'.format(sum(auc) / len(auc)))

print('='*100)


print('accuracy_test of sag : {}'.format(accuracy_test))
print('-'*100)
print('f1_test of sag : {}'.format(f1_test))
print('-'*100)
print('auc_test of sag : {}'.format(auc_test))

accuracy of sag : 0.8928753180661578
----------------------------------------------------------------------------------------------------
f1 of sag : 0.9353690925482201
----------------------------------------------------------------------------------------------------
auc of sag : 0.8581790049618219
accuracy_test of sag : [0.8952211338676165]
----------------------------------------------------------------------------------------------------
f1_test of sag : [0.9363619974761132]
----------------------------------------------------------------------------------------------------
auc_test of sag : [0.8538376771943298]


---

#### feature_importances

In [13]:
#지수표현식 삭제
pd.options.display.float_format = '{:.5f}'.format

rdmforest_clf_morph.feature_importances_

array([3.46529831e-03, 3.76632517e-05, 6.19321345e-03, ...,
       1.10279497e-04, 0.00000000e+00, 1.68918456e-05])

In [14]:
vectorizer_tfidf_morph.get_feature_names_out()

array(['10', '100', '100원', ..., '힘드네요', '힘든', '힘들어요'], dtype=object)

In [15]:
#지수표현식 삭제
pd.options.display.float_format = '{:.5f}'.format

#feature_importances
df_fture_importn = pd.DataFrame(rdmforest_clf_morph.feature_importances_, index=vectorizer_tfidf_morph.get_feature_names_out())
df_fture_importn = df_fture_importn.rename(columns={0:'feature_importances'})
df_fture_importn = df_fture_importn.sort_values(by = 'feature_importances', ascending=False)
df_fture_importn.head(40)

,feature_importances
업데이트,0.08049
장바구니,0.05805
오류,0.05120
가입,0.04509
문의,0.03193
광고,0.03017
결제,0.02346
삭제,0.02237
했는데,0.01785
그냥,0.01617


#### feature_importances는 부호구분이 안돼서, 해당 단어가 긍정 or 부정 리뷰 중 어느 리뷰에 영향력이 있는지 알 수 없다.

---

#### 새로운 리뷰 생성
#### 문장이 말이 되는지 않되는지는 상관이 없다.(벡터화는 문장의 단어 순서를 고려하지 않고 빈도수만 고려하기 때문이다.)

In [16]:
df_keywords_morph = pd.read_csv('keywords_morph_l2.csv').set_index(keys='Unnamed: 0', drop=True)
df_keywords_morph

,coefficient
Unnamed: 0,
사용,2.83171
좋은,2.75315
편하고,2.73994
최고,2.55686
편해요,2.50042
...,...
오류,-2.88598
장바구니,-2.92154
가입,-2.92890


In [21]:
#positive coefficient
df_keywords_morph_pos = df_keywords_morph[df_keywords_morph['coefficient'] > 0]
df_keywords_morph_pos

,coefficient
Unnamed: 0,
사용,2.83171
좋은,2.75315
편하고,2.73994
최고,2.55686
편해요,2.50042
...,...
8시,0.00372
역시,0.00343
있네요,0.00242


In [62]:
#positive words list
positive_words = list(df_keywords_morph_pos.index)
positive_words

['사용',
 '좋은',
 '편하고',
 '최고',
 '편해요',
 '빠르고',
 '만족합니다',
 '편리하고',
 '좋습니다',
 '아주',
 '신선하고',
 '빠른',
 '있어',
 '만족',
 '좋고',
 '감사합니다',
 '애용',
 '자주',
 '편하게',
 '항상',
 '정말',
 '있어서',
 '편리합니다',
 '편리해요',
 '포장',
 '쿠폰',
 '할인',
 '이용',
 '신선한',
 '다양한',
 '좋음',
 '만족해요',
 '깔끔하고',
 '신세계',
 '보기',
 '장보기',
 '매우',
 '편리하게',
 '마음',
 '너무',
 '들어요',
 '컬리',
 '편함',
 '새벽',
 '있습니다',
 '없이',
 '제품',
 '빨라요',
 '소량',
 '신선해요',
 '쉽게',
 '샛별',
 '편리',
 '요즘',
 '다양해서',
 '재료',
 '편리함',
 '간편하고',
 '품질',
 '쇼핑',
 '배송',
 '있었으면',
 '편하네요',
 '편리한',
 '맛있어요',
 '감사',
 '보는',
 '합니다',
 '보다',
 '신선해서',
 '중독',
 '좋겠어요',
 '좋아서',
 '혜택',
 '편합니다',
 '많아요',
 '많고',
 '빨리',
 '편해서',
 '환경',
 '클릭',
 '생활',
 '저렴하고',
 '먹거리',
 '신선하게',
 '쓰고',
 '편리하네요',
 '높은',
 '좋다',
 '간편하게',
 '받아',
 '품목',
 '식품',
 '깔끔한',
 '코로나',
 '라서',
 '한눈',
 '직관',
 '이에요',
 '특히',
 '감동',
 '퀄리티',
 '번창',
 '감사해요',
 '믿고',
 '다이어트',
 '간편해요',
 '신선도',
 '맛집',
 '기대',
 '고맙습니다',
 '같아요',
 '들어가',
 '진작',
 '구성',
 '편리해서',
 '되면',
 '비싸긴',
 '좋았습니다',
 '신속한',
 '아침',
 '단점',
 '에만',
 '조회',
 '싸게',
 '걱정',
 '있지만',
 '처음',
 '카테고리',
 

In [79]:
#positive_words에서 중복을 허용해서 5개 랜덤 뽑아서 문장 만들기(3000개)
pos_sentences = {}

for i in range(9000): 
    pos_sentence = [random.choice(positive_words) for i in range(5)]
    
    pos_sentences[' '.join(pos_sentence)] = 1

In [64]:
pos_sentences

{'맛있어요 시작 시작 가지 아기': 1,
 '시키면 역시 신선한 단어 싸게': 1,
 '빨라요 라면 대해 사서 평가': 1,
 '짱짱 편해서 5천 빨라 입장': 1,
 '되어있어요 기대 확실히 확대 클릭': 1,
 '하러 다양 많으면 비싸지만 기억': 1,
 '보장 일주일 만족스럽습니다 등록 아래': 1,
 '있다면 확실히 보통 구성 건강한': 1,
 '같아 맛있고 이며 편리한 않아': 1,
 '신선한 자주 된다면 아쉬운 일찍': 1,
 '스트레스 드라이아이스 쿠폰 싶네요 빼고': 1,
 '팔아서 높은 없지만 없으면 없이': 1,
 '시켜 이야 받아 없이 가능해서': 1,
 '특별한 보기 가지 힘든 있으면': 1,
 '리스트 높은 주소지 적극 선물': 1,
 '가기 신속하고 거나 줄어들었고 편입': 1,
 '만족합니다 아무튼 만족도 마음 고를': 1,
 '담아 대체로 들어가서 배송 감사해요': 1,
 '최고 필요한 하러 경쟁 간편하고': 1,
 '깔끔한 신선도 해주면 8시 하러': 1,
 '최근 전반 빼고 온라인 아니지만': 1,
 '샐러드 빼고 비싸지만 되니 쓰고': 1,
 '담날 편해서 작은 사면 주고': 1,
 '만큼 시작 좋긴 분실 가게': 1,
 '담날 편리합니다 해주시 마트 칭찬': 1,
 '정확한 겁니다 대해 수단 적극': 1,
 '편리하게 일찍 상태 올려주세요 있는데': 1,
 '대신 자취 되었으면 다양하게 되니': 1,
 '가게 하시길 생겼으면 많이 만들어': 1,
 '쉽게 상당히 깔끔합니다 드립니다 심플': 1,
 '이지 장보고 편한 안심 빨리': 1,
 '맛있고 깔끔합니다 기원 먹어 다양한': 1,
 '됐는데 시킬 나름 식품 싱싱한': 1,
 '경쟁 좋게 식품 맛있고 편할': 1,
 '정렬 으로도 드라이아이스 디자인 있지만': 1,
 '결정 신선한 결정 깨끗하고 신세계': 1,
 '괜찮고 빼고는 줘요 무척 조절': 1,
 '은근 아침 노력 적극 되어있고': 1,
 '먹고 비교 들어서 주셨으면 샛별': 1,
 '깨끗하고 위해 엄마 같아요 친

In [80]:
#새로운 positive 리뷰
df_review_new_pos = pd.DataFrame(pd.Series(pos_sentences)).reset_index().rename(columns={'index': 'review', 0:'label'})
df_review_new_pos

,review,label
0,편하네요 받으니 주소지 꼼꼼히 칭찬,1
1,가능하면 가게 유튜브 만족도 아침,1
2,간편하게 편해서 배송비 신뢰 은근,1
3,먹고 전이 보임 편리한 고기,1
4,로만 됐어요 번창 빠르네요 다양한,1
...,...,...
8995,가격 요리 만족도 활용 단점,1
8996,아니지만 종종 좋겠어요 있었는데 검색어,1
8997,코로나 감사합니다 사랑 먹거리 유용하게,1
8998,만족합니다 있네요 만큼 거기 들어요,1


In [81]:
#negative coefficient
df_keywords_morph_neg = df_keywords_morph[df_keywords_morph['coefficient'] < 0]
df_keywords_morph_neg

,coefficient
Unnamed: 0,
많네요,-0.00050
일도,-0.00082
사과,-0.00087
없고요,-0.00126
쇼핑몰,-0.00148
...,...
오류,-2.88598
장바구니,-2.92154
가입,-2.92890


In [82]:
#negative words list
negative_words = list(df_keywords_morph_neg.index)
negative_words

['많네요',
 '일도',
 '사과',
 '없고요',
 '쇼핑몰',
 '개월',
 '개수',
 '관심',
 '되었다고',
 '또는',
 '송이',
 '된다는',
 '지는',
 '하려고요',
 '도대체',
 '거래',
 '갑니다',
 '만에',
 '까지는',
 '시대',
 '평소',
 '힘드네요',
 '시국',
 '전지현',
 '가요',
 '물품',
 '같은데',
 '줬으면',
 '많습니다',
 '방식',
 '봅니다',
 '씁니다',
 '정상',
 '하길',
 '깔고',
 '가장',
 '받기',
 '들어갈',
 '맞는',
 '많은데',
 '사이트',
 '좋겠네요',
 '식료품',
 '시도',
 '내용물',
 '오면',
 '보라색',
 '음식',
 '대표',
 '하시는',
 '없는',
 '딱히',
 '지웠다',
 '무료',
 '괜찮은',
 '목록',
 '보이는',
 '대요',
 '걸리고',
 '저만',
 '완전',
 '보단',
 '나와서',
 '오래',
 '가능',
 '유기농',
 '시키는',
 '급하게',
 '좋을듯',
 '소리',
 '나오는데',
 '먹기',
 '이라',
 '체크',
 '반찬',
 '가서',
 '봐요',
 '제외',
 '가능하게',
 '항목',
 '수량',
 '좋지만',
 '근데',
 '않지만',
 '넣어',
 '이미',
 '괜찮네요',
 '나오네요',
 '이란',
 '필요할',
 '보면',
 '초기',
 '남깁니다',
 '나오면',
 '으로는',
 '포인트',
 '제거',
 '써서',
 '매일',
 '용량',
 '가고',
 '버튼',
 '받았어요',
 '9시',
 '10시',
 '직접',
 '유통',
 '반복',
 '바랍니다',
 '이라서',
 '물량',
 '신청',
 '안된다고',
 '프레',
 '해주세요',
 '냉동식품',
 '같아서',
 '한테',
 '비닐',
 '크게',
 '주말',
 '그것',
 '유지',
 '있나요',
 '자고',
 '있고요',
 '진심',
 '똑같은',
 '상자',
 '있다고',
 '지금',
 '다가',
 '수거'

In [83]:
#positive_words에서 중복을 허용해서 5개 랜덤 뽑아서 문장 만들기(3000개)
neg_sentences = {}

for i in range(1000): 
    neg_sentence = [random.choice(negative_words) for i in range(5)]
    
    neg_sentences[' '.join(neg_sentence)] = 0

In [72]:
neg_sentences

{'짜증 참고 옵션 시대 가는': 0,
 '개판 계속 100원 런가 하자': 0,
 '반찬 경쟁력 없는데 그렇게 봐요': 0,
 '문의 에러 보여 지웠다 낚시': 0,
 '회원 누르면 네이버 담아도 평소': 0,
 '있게 대체 대비 되는데 이고': 0,
 '안됩니다 괜찮네요 마찬가지 장점 넣어': 0,
 '없다 에서는 고요 낚시 하세요': 0,
 '누락 평점 시국 텐데 아쉬워요': 0,
 '실수 안됨 좋을듯 통장 오랜': 0,
 '고생 준다고 소비 밖에 화가': 0,
 '주는 장바구니 구분 센터 아이콘': 0,
 '마케팅 편이 아예 경험 거지': 0,
 '거지 반찬 11 시키는 주는': 0,
 '상담사 없나요 완전 있나요 사라': 0,
 '깔고 데이터 알람 건지 이틀': 0,
 '구분 있는지 항목 판매 중요한': 0,
 '오네요 최신 바랍니다 되지 주말': 0,
 '보이네요 완전 변경 늦게 운영': 0,
 '상온 네이버 에선 개발 노트': 0,
 '잘못 장애 맞는 많았으면 뜨는데': 0,
 '종료 정상 일반 날짜 하시는': 0,
 '않을 탈퇴 아직도 무료 이라는': 0,
 '프레 자체 않아서 기본 짜증': 0,
 '했지만 예전 오지 홍보 진행': 0,
 '최소 아파트 이라도 해결 작성': 0,
 '이었는데 다가 송이 가는 나오면': 0,
 '이고 완료 추가 도착 노트': 0,
 '같은데 불편하네요 이랑 괜찮은 기한': 0,
 '이틀 안됨 발생 속도 하지': 0,
 '그거 그러네요 받았어요 보고 자체': 0,
 '모르겠어요 도대체 마감 들어가면 세분': 0,
 '쓰려고 상세 괜찮은 11 접속': 0,
 '나요 된다 부분 그냥 건가': 0,
 '필터 일부 하려고요 괜찮네요 처리': 0,
 '말고 진짜 점검 이니 추가': 0,
 '인지 찾는 부담 한정 되지': 0,
 '아직도 이면 장바구니 품절 안녕하세요': 0,
 '있나요 다양하지 가능 인가요 아니네요': 0,
 '상황 대가 시도 업체 한두': 0,
 '나오네요 예정 받는 체크 정리': 0,
 '송이 이라고 표기

In [84]:
#새로운 negative 리뷰
df_review_new_neg = pd.DataFrame(pd.Series(neg_sentences)).reset_index().rename(columns={'index': 'review', 0:'label'})
df_review_new_neg

,review,label
0,다운 시장 테이프 한두 수가,0
1,담고 있다고 식료품 보지 구분,0
2,서울 요청 문제 오래 공지,0
3,물건 오지 대기 아예 최신,0
4,표기 정확 이라 찾을 이라서,0
...,...,...
995,서울 이벤트 같음 예전 기업,0
996,수거 개발 이벤트 11시 맞는,0
997,마케팅 비번 괜찮은 건데 시키는,0
998,보단 받은 뜨고 이랑 물건,0


In [85]:
#새로운 positive negative 리뷰 합치기
df_review_new = pd.concat([df_review_new_pos, df_review_new_neg])
df_review_new

,review,label
0,편하네요 받으니 주소지 꼼꼼히 칭찬,1
1,가능하면 가게 유튜브 만족도 아침,1
2,간편하게 편해서 배송비 신뢰 은근,1
3,먹고 전이 보임 편리한 고기,1
4,로만 됐어요 번창 빠르네요 다양한,1
...,...,...
995,서울 이벤트 같음 예전 기업,0
996,수거 개발 이벤트 11시 맞는,0
997,마케팅 비번 괜찮은 건데 시키는,0
998,보단 받은 뜨고 이랑 물건,0


In [87]:
#새로운 리뷰들 벡터화 후 예측

#skfold
skfold = StratifiedKFold(n_splits=5)

#best randomforest
rdmforest_clf_morph = RandomForestClassifier(max_depth=35, 
                                             max_features=0.1, 
                                             min_samples_leaf=1, 
                                             min_samples_split=6, 
                                             n_estimators=400)


#train(fit)은 기존의 review data로 한다.
for train_idx, validation_idx in skfold.split(X_morph_train, y=y_morph_train):
        
    X_train = X_morph_train[train_idx]
    X_validation = X_morph_train[validation_idx]
        
    y_train = y_morph_train.iloc[train_idx]
    y_validation = y_morph_train.iloc[validation_idx]
        
    #fit_predict
    rdmforest_clf_morph.fit(X_train, y_train)
    
    
#새로운 리뷰 test data
contents_for_vectorize_new = df_review_new['review']


X_test_new = vectorizer_tfidf_morph.fit_transform(contents_for_vectorize_new)
y_test_new = df_review_new['label']


#새로운 리뷰 predict
pred_test_new = rdmforest_clf_morph.predict(X_test_new)


#test_data accuracy, f1, auc
accuracy_test.append(accuracy_score(pred_test_new, y_test_new))
f1_test.append(f1_score(pred_test_new, y_test_new))
auc_test.append(roc_auc_score(pred_test_new, y_test_new))

ValueError: X has 709 features, but RandomForestClassifier is expecting 1367 features as input.

#### 실제 리뷰 데이터로 fit을 하고, 새롭게 생성한 리뷰로 predict을 하는 것은 실행이 안된다. 
#### 실제 리뷰 데이터를 벡터화해서 생성된 feature와 새로운 리뷰를 벡터화해서 생성된 feature의 개수가 다르기 때문이다.
#### 그렇다고 새롭게 생성된 리뷰로 fit하고 predict하는 것은 그냥 학습한 것으로 예측하는 것이기 때문에 의미가 없다.